In [1]:
import numpy as np
import pandas as pd
import scipy
import pickle
import flexcode
from flexcode.regression_models import NN, RandomForest 

import matplotlib.pyplot as plt

import local_conformal as lc

In [2]:
%matplotlib inline

Variation:

- `n_size`: 100, 316, 1000, 3162, 10000
- `model_type`: [NN, RandomForest]

Data Sizes (Training/CDE-Tune/Calibrate):

*For Izbicki's FlexCode approach we need tune the model (say .6/.4) so we give more to the training (looks like they have to calibrate the actual model... -> not sure if this effects things...)
- Izbicki: 2n/0/1n
- Us: 1.5n/.75n/.75n




# WARNING:
**I don't think FlexCode's interface with RF is working...**

In [3]:
# nsize range
power_range = np.array([2,2.5,3,3.5,4])
size_range = np.array(10**power_range, dtype = np.int)
test_sim_size = 300
sim_index = np.arange(20)[0]

In [4]:
my_generate_data = lc.generate_data_b2
my_generate_data_cond = lc.generate_data_b2_cond

In [47]:
s = size_range[0]
model_type = np.array(["NN","RF"])[0]


In [48]:
# data create ------------------

##### testing data -------------

x_test_range = np.linspace(-2,2, num = 201)
y_test_list = my_generate_data_cond(x_test_range, test_sim_size)

##### data for models ----------
x,y = my_generate_data(size = 3*s)

iz_splits_prop = np.array([2*.6, 2*.4, 0 , 1])/3
us_splits_prop = np.array([1.5*.6, 1.5*.4, .75, .75])/3

iz_splits = np.random.choice(4, 3*s, p=iz_splits_prop)
us_splits = np.random.choice(4, 3*s, p=us_splits_prop)


x_train_iz_1 = x[iz_splits == 0]
y_train_iz_1 = y[iz_splits == 0]
x_train_iz_2 = x[iz_splits == 1]
y_train_iz_2 = y[iz_splits == 1]

x_calibrate_iz = x[iz_splits == 3] 
y_calibrate_iz = y[iz_splits == 3]

x_train_us_1 = x[us_splits == 0] 
y_train_us_1 = y[us_splits == 0]
x_train_us_2 = x[us_splits == 1] 
y_train_us_2 = y[us_splits == 1]

x_cde_tune_us = x[us_splits == 2] 
y_cde_tune_us = y[us_splits == 2]

x_calibrate_us = x[us_splits == 3] 
y_calibrate_us = y[us_splits == 3]

# 1. model fits ---------

##### Izbicki --------
if model_type == "NN":
    model_iz = flexcode.FlexCodeModel(NN, max_basis=31, basis_system="cosine",
                                   regression_params={"k":20})
elif model_type == "RF":
    model_iz = flexcode.FlexCodeModel(RandomForest, max_basis=31, basis_system="cosine",
                                   regression_params={"k":20})

model_iz.fit(x_train_iz_1,y_train_iz_1)
model_iz.tune(x_train_iz_2, y_train_iz_2)


##### Us --------
if model_type == "NN":
    model_us = flexcode.FlexCodeModel(NN, max_basis=31, basis_system="cosine",
                                   regression_params={"k":20})
elif model_type == "RF":
    model_us = flexcode.FlexCodeModel(RandomForest, max_basis=31, basis_system="cosine",
                                   regression_params={"k":20}) # Why no work? :(, complains about parameters...

model_us.fit(x_train_iz_1, y_train_iz_1)
model_us.tune(x_train_iz_2, y_train_iz_2)


# 2. Our CDE correction ------------



# 3. Profile grouping --------------



# TODO:

1. make sure we have the grouping approach for Izbicki's approach...

*Thought*: *Izbicki's profile should use the `hpd_coverage` function internally - I think.*

# Overall:

1. Us: estimate grouping models based on CDE-tune data
2. Grouping approaches (`profile_grouping` function can be used)
3. Testing: define grid along X space, sample 300 Y values, and then see the proportion contained at levels .95,.9,.8,.6

## data collection:

| sim index |`n` Size |Model Type | X value| expected confidence | actual attainment |
|-|-|-|-|-|-|
|1,...,20| 100, 316, 1000, 3162, 10000| RF + ??, NN + MLP| range between -2,2 | .95,.9,.8,.6 | **???** |


## How to visualize?

